In [98]:
import boto3
import botocore
import sagemaker
import sys


bucket = 'sciforma-performance-data'   # <--- specify a bucket you have access to
prefix = 'sagemaker/rcf-benchmarks'
execution_role = sagemaker.get_execution_role()


# check if the bucket exists
try:
    boto3.Session().client('s3').head_bucket(Bucket=bucket)
except botocore.exceptions.ParamValidationError as e:
    print('Hey! You either forgot to specify your S3 bucket'
          ' or you gave your bucket an invalid name!')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == '403':
        print("Hey! You don't have permission to access the bucket, {}.".format(bucket))
    elif e.response['Error']['Code'] == '404':
        print("Hey! Your bucket, {}, doesn't exist!".format(bucket))
    else:
        raise
else:
    print('Training input/output will be stored in: s3://{}/{}'.format(bucket, prefix))


import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()
data_key = 'performanceDataWithRole_Loc_mag_project_scheduler.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

performanceData = pd.read_csv(data_location)
performanceData.drop(columns='Unnamed: 0', inplace=True)
performanceData['ELAPSED'] = performanceData['ELAPSED'].div(1000) 
performanceData.drop(performanceData[performanceData['ELAPSED']>1500].index, inplace=True)
performanceData.head()

Training input/output will be stored in: s3://sciforma-performance-data/sagemaker/rcf-benchmarks


,STARTDATE,ELAPSED,USERID,EVENTID,EVENT_DESC,WORKSPACEID,WS_DESC,TRANSACTIONTYPE,CORE_ID,HR_ORGANIZATION,LOCATION,USER_ROLE,MAG_CODE
0,2019-04-06 16:05:37,2.329,2,xkJAb1CZP,Project Scheduler,,Exception,1,NaN,Entire Organization,NaN,Super User,NaN
1,2019-04-06 16:21:57,3.157,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,The Demo Org.SP Demo Org,TX30,_Program Manager,RB4
2,2019-04-06 16:26:51,4.568,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,The Demo Org.SP Demo Org,TX30,_Program Manager,RB4
3,2019-04-06 16:34:07,2.451,2493687,xkJAb1CZP,Project Scheduler,,Exception,1,dpmosp,The Demo Org.SP Demo Org,TX30,_PMO+RM,RB4
4,2019-04-06 17:42:31,6.817,8929278,xkJAb1CZP,Project Scheduler,,Exception,1,NXA24209,HPMS.STI,IN-BLR01-s1,_Program Manager,RC9


In [101]:
performanceData.ELAPSED.mean()

ELAPSED            5.114573e+00
USERID             2.467798e+07
TRANSACTIONTYPE    8.355369e-01
dtype: float64

In [102]:
performanceData.ELAPSED.median()

performanceData.ELAPSED.describe()
#performanceData["time_bin"]= ['0_to_1' if ((x <= 1) and (x >0)) else '2_to_more' for x in performanceData['ELAPSED']]  
    
def time_bin_method(df):
    if (0 < df['ELAPSED'] <= 2):
        return '0_to_2'
    elif (2 < df['ELAPSED'] <= 4):
        return '2_to_4'
    elif (4 < df['ELAPSED'] <= 6):
        return '4_to_6'
    elif (df['ELAPSED'] > 6):
        return '6_or_more'

def time_bin_method_pr(df):
    if (0 < df['ELAPSED'] <= 4.4):
        return '0_to_4.4'
    else :
        return '4.4_or_more'    
performanceData['time_bin'] = performanceData.apply(time_bin_method, axis = 1)
performanceData.head(3)

,STARTDATE,ELAPSED,USERID,EVENTID,EVENT_DESC,WORKSPACEID,WS_DESC,TRANSACTIONTYPE,CORE_ID,HR_ORGANIZATION,LOCATION,USER_ROLE,MAG_CODE,time_bin
0,2019-04-06 16:05:37,2.329,2,xkJAb1CZP,Project Scheduler,,Exception,1,NaN,Entire Organization,NaN,Super User,NaN,2_to_4
1,2019-04-06 16:21:57,3.157,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,The Demo Org.SP Demo Org,TX30,_Program Manager,RB4,2_to_4
2,2019-04-06 16:26:51,4.568,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,The Demo Org.SP Demo Org,TX30,_Program Manager,RB4,4_to_6


In [103]:
performanceData['HR_ORGANIZATION'] = 'org_' + performanceData['HR_ORGANIZATION'].astype(str)
performanceData['LOCATION'] = 'loc_' + performanceData['LOCATION'].astype(str)
performanceData['USER_ROLE'] = 'role_' + performanceData['USER_ROLE'].astype(str)
performanceData.head(3)

,STARTDATE,ELAPSED,USERID,EVENTID,EVENT_DESC,WORKSPACEID,WS_DESC,TRANSACTIONTYPE,CORE_ID,HR_ORGANIZATION,LOCATION,USER_ROLE,MAG_CODE,time_bin
0,2019-04-06 16:05:37,2.329,2,xkJAb1CZP,Project Scheduler,,Exception,1,NaN,org_Entire Organization,loc_nan,role_Super User,NaN,2_to_4
1,2019-04-06 16:21:57,3.157,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,org_The Demo Org.SP Demo Org,loc_TX30,role__Program Manager,RB4,2_to_4
2,2019-04-06 16:26:51,4.568,2493686,xkJAb1CZP,Project Scheduler,,Exception,1,wpmsp,org_The Demo Org.SP Demo Org,loc_TX30,role__Program Manager,RB4,4_to_6


In [104]:
#filters = ['4.4_or_more']
filters = ['4_to_6','6_or_more']
performance_subset = performanceData[performanceData.time_bin.isin(filters)]
#performance_subset = performanceData
#datasetApriori = performance_subset.drop(performance_subset.columns.difference(['HR_ORGANIZATION','LOCATION','USER_ROLE','time_bin']), 1)
datasetApriori = performance_subset.drop(performance_subset.columns.difference(['HR_ORGANIZATION','LOCATION','USER_ROLE']), 1)
datasetApriori.dropna(inplace = True)
datasetApriori = datasetApriori.values.tolist()
#print(datasetApriori)

In [105]:
# Apriori analysis
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_ary = te.fit(datasetApriori).transform(datasetApriori)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head(3)


,loc_AT-GRK01-s1,loc_BE-LEU01-s1,loc_BR-CPQ01,loc_CA-KAN01,loc_CN-BJS01,loc_CN-CDU01,loc_CN-SHA01-s1,loc_CN-SHA02,loc_CN-SZH02,loc_CN-TNJ02,...,role__PM+RM,role__PM+RM+Portfolio,role__PMO+RM,role__PMO+RM View Only,role__Product Engineering Manager,role__Product Line Marketer,role__Program Manager,role__Resource,role__Resource Controller,role__Resource Manager
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [88]:
performanceData['time_bin'].describe()

#apriori(df, min_support=0.05)

count      25594
unique         4
top       0_to_2
freq       13659
Name: time_bin, dtype: object

In [106]:
frequent_itemsets= apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.tail(10)

,support,itemsets,length
150,0.014723,"(org_HPMS.AAA, loc_NL-NYM01-s1, role__PMO+RM)",3
151,0.013682,"(loc_NL-NYM01-s1, org_HPMS.SIP, role__PMO+RM)",3
152,0.012121,"(loc_NL-NYM01-s1, org_T&O.TO-PT, role__PMO+RM)",3
153,0.037106,"(role__Core Team Support User, org_The Support...",3
154,0.014575,"(loc_RU-MOW02, role__Program Manager, org_T&O....",3
155,0.010782,"(org_HPMS.DN, loc_US-AUS01, role__PMO+RM)",3
156,0.040006,"(loc_US-AUS01, org_HPMS.MICR, role__PMO+RM)",3
157,0.010113,"(loc_US-CHD01, org_HPMS.AAA, role__PMO+RM)",3
158,0.030860,"(loc_US-CHD01, org_HPMS.RPS, role__PMO+RM)",3
159,0.011377,"(loc_nan, role__Core Team User, org_The Suppor...",3


In [107]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
#rules.to_csv("project_scheduler_rules_mlxtend.csv")

In [ ]:
confidence = rules['confidence'] > 0.8
lift = rules['lift'] > 20
#elderly = df['age'] > 50

rules= rules.sort_values(by=['confidence','lift'], ascending=False)
rules[confidence]

In [108]:

def unionizeFrozenset(rules):
        return frozenset.union(rules['antecedents'],rules['consequents'])

performanceData['time_bin'] = performanceData.apply(time_bin_method, axis = 1)



rules['Combined_frozenset']=rules.apply(unionizeFrozenset, axis = 1)

In [109]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Combined_frozenset
0,(org_HPMS.STI),(loc_AT-GRK01-s1),0.255800,0.147085,0.124256,0.485756,3.302550,0.086632,1.658580,"(org_HPMS.STI, loc_AT-GRK01-s1)"
1,(loc_AT-GRK01-s1),(org_HPMS.STI),0.147085,0.255800,0.124256,0.844793,3.302550,0.086632,4.794877,"(org_HPMS.STI, loc_AT-GRK01-s1)"
2,(loc_AT-GRK01-s1),(role_<No Rights>),0.147085,0.024465,0.013385,0.091001,3.719701,0.009787,1.073197,"(loc_AT-GRK01-s1, role_<No Rights>)"
3,(role_<No Rights>),(loc_AT-GRK01-s1),0.024465,0.147085,0.013385,0.547112,3.719701,0.009787,1.883282,"(loc_AT-GRK01-s1, role_<No Rights>)"
4,(loc_BE-LEU01-s1),(org_HPMS.SIP),0.013682,0.051755,0.013608,0.994565,19.216829,0.012900,174.477097,"(loc_BE-LEU01-s1, org_HPMS.SIP)"


In [110]:
uninionizedRules = rules.drop(columns=['antecedents','consequents'])
uninionizedRules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000,222.000000
mean,0.100447,0.100447,0.022073,0.483747,9.000742,0.016634,inf
std,0.107175,0.107175,0.016825,0.353378,13.077558,0.012392,NaN
min,0.010262,0.010262,0.010113,0.027512,2.010766,0.005158,1.014221
25%,0.024465,0.024465,0.011637,0.152997,2.621984,0.008951,1.138964
50%,0.060529,0.060529,0.014723,0.405011,3.938322,0.011221,1.553361
75%,0.103287,0.103287,0.028703,0.848485,9.671987,0.022901,5.317952
max,0.372992,0.372992,0.124256,1.000000,81.012048,0.086632,inf


In [111]:
uninionizedRules = uninionizedRules.groupby(['Combined_frozenset']).mean()


In [112]:
uninionizedRules.reset_index(inplace=True)
uninionizedRules.head()

,Combined_frozenset,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(org_HPMS.STI, loc_AT-GRK01-s1)",0.201443,0.201443,0.124256,0.665274,3.302550,0.086632,3.226729
1,"(org_HPMS.STI, role__PM+RM)",0.211258,0.211258,0.088117,0.436511,2.066246,0.045471,1.424847
2,"(org_T&O.TO-FO, role__PMO+RM)",0.196721,0.196721,0.020077,0.517823,2.632275,0.012450,17.760351
3,"(role__Core Team Support User, org_The Support...",0.043129,0.043129,0.037106,0.877458,20.344932,0.035282,inf
4,"(role__Core Team User, org_The Support IT.A Su...",0.030748,0.030748,0.012046,0.610493,19.854693,0.011440,20.384239


In [163]:
uninionizedRules.to_csv("project_resource_Union_rules_mlxtend.csv")

In [113]:
import re
uniqueRole = performanceData.USER_ROLE.unique()
uniqueHROrg = performanceData.HR_ORGANIZATION.unique()
uniqueLOCATION = performanceData.LOCATION.unique()

def search(list_v,subs):
    for val in list_v:
        if val.startswith(subs):
            return val

def calculatemean(df):
    #print(df['Combined_frozenset'])
    role_in_set = [x for x in df['Combined_frozenset'] if re.search('role_', x)]
    org_in_set = [x for x in df['Combined_frozenset'] if re.search('org_', x)]
    loc_in_set = [x for x in df['Combined_frozenset'] if re.search('loc_', x)]
    if len(role_in_set) > 0 :
        user_role_filter = role_in_set
       # print("role_in_set")
    else :
        user_role_filter = uniqueRole
    if len(org_in_set)>0 :
        org_filter = org_in_set
        #print("org_in_set")
    else :
        org_filter = uniqueHROrg
    if len(loc_in_set)>0 :
        loc_filter = loc_in_set
       # print("loc_in_set")
    else :
        loc_filter = uniqueLOCATION
        
    filteredPerformance = performanceData[(performanceData.USER_ROLE.isin(user_role_filter)) 
                                          & (performanceData.HR_ORGANIZATION.isin(org_filter)) 
                                          & (performanceData.LOCATION.isin(loc_filter))]
    #return str(filteredPerformance.ELAPSED.mean())+', '+str(filteredPerformance.ELAPSED.median())+', '+filteredPerformance.size()
    return str(round(filteredPerformance.ELAPSED.mean(),2))+','+str(round(filteredPerformance.ELAPSED.median(),2))+','+str(len(filteredPerformance))

uninionizedRules['time_elapsed_aggregate'] = uninionizedRules.apply(calculatemean, axis = 1)
uninionizedRules.tail(3)

,Combined_frozenset,antecedent support,consequent support,support,confidence,lift,leverage,conviction,time_elapsed_aggregate
0,"(org_HPMS.STI, loc_AT-GRK01-s1)",0.201443,0.201443,0.124256,0.665274,3.302550,0.086632,3.226729,"4.97,4.02,3301"
1,"(org_HPMS.STI, role__PM+RM)",0.211258,0.211258,0.088117,0.436511,2.066246,0.045471,1.424847,"5.31,4.03,2335"
2,"(org_T&O.TO-FO, role__PMO+RM)",0.196721,0.196721,0.020077,0.517823,2.632275,0.012450,17.760351,"4.79,2.87,1350"
3,"(role__Core Team Support User, org_The Support...",0.043129,0.043129,0.037106,0.877458,20.344932,0.035282,inf,"5.72,3.6,1242"
4,"(role__Core Team User, org_The Support IT.A Su...",0.030748,0.030748,0.012046,0.610493,19.854693,0.011440,20.384239,"6.21,3.84,339"
5,"(org_HPMS.AAA, role__PMO+RM, loc_AT-GRK01-s1)",0.138329,0.138329,0.014649,0.342895,2.395920,0.008489,6.120073,"4.98,3.51,534"
6,"(org_HPMS.STI, loc_AT-GRK01-s1, role_<No Rights>)",0.096842,0.096842,0.011972,0.392287,4.167530,0.009030,2.550112,"5.1,4.03,316"
7,"(org_HPMS.STI, role__PM+RM, loc_AT-GRK01-s1)",0.138645,0.138645,0.019631,0.495119,3.571127,0.014134,4.831399,"4.79,3.67,674"
8,"(org_HPMS.STI, loc_AT-GRK01-s1, role__PMO+RM)",0.135262,0.135262,0.056291,0.558415,4.397634,0.042274,2.887826,"5.62,4.55,1172"
9,"(role__Program Manager, org_HPMS.STI, loc_AT-G...",0.130651,0.130651,0.035098,0.428136,3.240138,0.024180,2.865103,"4.55,3.9,1009"


In [114]:
uninionizedRules.to_csv("project_scheduler_set_w_time_aggre.csv")